# Optimize Interface Film Position

This notebook optimizes the position of a film in an interface structure by:
1. Evaluating energies across an x-y grid of positions
2. Finding the optimal position that minimizes interaction energy
3. Visualizing the energy landscape

## 1. Set up the environment


In [ ]:
# Parameters for optimization
GRID_SIZE = (20, 20)  # Resolution of the x-y grid
GRID_RANGE_X = (-0.5, 0.5)  # Range to search in x direction (in crystal coordinates)
GRID_RANGE_Y = (-0.5, 0.5)  # Range to search in y direction (in crystal coordinates)
USE_CARTESIAN = False  # Whether to use Cartesian coordinates
SHADOWING_RADIUS = 2.5  # Radius for detecting surface atoms (in Angstroms)


## 2. Import required packages


In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from mat3ra.made.material import Material



## 3. Define visualization functions


In [ ]:
def plot_energy_landscape(xy_matrix, energy_matrix, optimal_position=None):
    """
    Create a 3D surface plot of the energy landscape.
    
    Args:
        xy_matrix (List[np.ndarray]): X and Y coordinate matrices
        energy_matrix (np.ndarray): Matrix of energy values
        optimal_position (tuple, optional): The optimal (x,y) position to highlight
    """
    x_vals, y_vals = xy_matrix
    X, Y = np.meshgrid(x_vals, y_vals)
    
    # Create the 3D surface plot
    fig = go.Figure(data=[
        go.Surface(x=X, y=Y, z=energy_matrix, colorscale='Viridis')
    ])
    
    # Add optimal position marker if provided
    if optimal_position is not None:
        x_opt, y_opt = optimal_position[0], optimal_position[1]
        z_opt = np.min(energy_matrix)
        fig.add_trace(go.Scatter3d(
            x=[x_opt], y=[y_opt], z=[z_opt],
            mode='markers',
            marker=dict(size=8, color='red'),
            name='Optimal Position'
        ))
    
    fig.update_layout(
        title='Interface Energy Landscape',
        scene=dict(
            xaxis_title='X Position',
            yaxis_title='Y Position',
            zaxis_title='Energy'
        ),
        width=800,
        height=800
    )
    
    fig.show()


def plot_energy_heatmap(xy_matrix, energy_matrix, optimal_position=None):
    """Create a 2D heatmap of the energy landscape."""
    x_vals, y_vals = xy_matrix

    fig = go.Figure(data=go.Heatmap(
        x=x_vals,
        y=y_vals,
        z=energy_matrix,
        colorscale='Viridis',
        colorbar=dict(title='Energy')
    ))

    if optimal_position is not None:
        x_opt, y_opt = optimal_position[0], optimal_position[1]
        fig.add_trace(go.Scatter(
            x=[x_opt],
            y=[y_opt],
            mode='markers',
            marker=dict(size=12, color='red', symbol='x'),
            name='Optimal Position'
        ))

    fig.update_layout(
        title='Interface Energy Heatmap',
        xaxis_title='X Position',
        yaxis_title='Y Position',
        width=800,
        height=600
    )

    fig.show()



## 6. Load and optimize material


In [ ]:
from mat3ra.made.tools.build.interface import get_optimal_film_displacement
# Import required packages 
from utils.visualize import visualize_materials
from mat3ra.made.tools.build.interface import interface_displace_part
from mat3ra.made.tools.calculate.calculators import InterfaceMaterialCalculator
from mat3ra.made.tools.optimize import evaluate_calculator_on_xy_grid
from utils.jupyterlite import get_materials, set_materials
import numpy as np
import plotly.graph_objects as go

# Parameters
GRID_SIZE = (20, 20)
GRID_RANGE_X = (-0.0, 1.0)
GRID_RANGE_Y = (-0.0, 1.0)
USE_CARTESIAN = False



# Get the material
materials = get_materials(globals())
interface_material = materials[0]
interface_material = interface_displace_part(
    interface_material,
    displacement=[0.24, 1.1,0],
    use_cartesian_coordinates=USE_CARTESIAN
)
print("Material labels:", interface_material.basis.labels)

# Initialize calculator
calculator = InterfaceMaterialCalculator()

# Calculate energy landscape
xy_matrix, energy_matrix = evaluate_calculator_on_xy_grid(
    material=interface_material,
    calculator_function=calculator.get_energy,
    modifier=interface_displace_part,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)

optimal_displacement = get_optimal_film_displacement(material=interface_material, calculator=calculator,   
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN)
                                                     


In [ ]:

# Plot energy landscape
plot_energy_heatmap(xy_matrix, energy_matrix, optimal_position=optimal_displacement)
plot_energy_landscape(xy_matrix, energy_matrix, optimal_position=optimal_displacement)
# Create optimized material
optimized_material = interface_displace_part(
    interface_material,
    displacement=optimal_displacement,
    use_cartesian_coordinates=USE_CARTESIAN
)

# Visualize materials
print("\nVisualization of original and optimized materials:")
visualize_materials([interface_material, optimized_material], repetitions=[3, 3,1])
visualize_materials([interface_material, optimized_material], repetitions=[3,3,1], rotation='-90x')

print(f"\nOptimal displacement: {optimal_displacement}")

# Save optimized material (uncomment to use)
# set_materials(optimized_material)